In [1]:
import pandas as pd
import numpy as np

In [61]:
def compute_rho_eb_te_cr(data_name):
    cr_df = pd.read_csv('../results/cr/all_cr.csv')
    cr_df = cr_df[cr_df.data == data_name]
    te_df = pd.read_csv('../results/te/transformation_error.csv')
    te_df = te_df[te_df.data == data_name]
    te_df = te_df[te_df.metric == 'nrmse']
    cr_df = cr_df.rename({'error_bound': 'eb', 'gzip': 'cr'}, axis=1)
    te_df = te_df.rename({'decompression error': 'te'}, axis=1)
    cr_df = cr_df[cr_df.eb.isin(te_df.eb)]
    cr_df.set_index(['eb', 'compression'], inplace=True)
    te_df.set_index(['eb', 'compression'], inplace=True)
    cr_df.drop(['data'], axis=1, inplace=True)
    te_df.drop(['data'], axis=1, inplace=True)
    joined = cr_df.join(te_df)
    joined.reset_index(inplace=True)
    display(joined.groupby('compression').corr())
    return joined

In [62]:
joined = compute_rho_eb_te_cr('aus')

eb        cr        te
compression                                 
pmc         eb  1.000000  0.785108  0.953444
            cr  0.785108  1.000000  0.579409
            te  0.953444  0.579409  1.000000
swing       eb  1.000000  0.849640  0.979876
            cr  0.849640  1.000000  0.771676
            te  0.979876  0.771676  1.000000
sz          eb  1.000000  0.779702  0.988782
            cr  0.779702  1.000000  0.701564
            te  0.988782  0.701564  1.000000

In [66]:
import statsmodels.formula.api as smf
for comp in ['pmc', 'swing', 'sz']:
    print(comp)
    model = smf.ols(formula='cr ~ te', data=joined[(joined['compression'] == comp)]).fit()
    display(model.summary())

pmc


D:\Anaconda\envs\dart\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     cr   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.275
Method:                 Least Squares   F-statistic:                     5.559
Date:                Tue, 12 Sep 2023   Prob (F-statistic):             0.0380
Time:                        18:11:54   Log-Likelihood:                -115.76
No. Observations:                  13   AIC:                             235.5
Df Residuals:                      11   BIC:                             236.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -968.9107    948.828     -1.021      0.329   -3057.268    1119.446
te          2.652e+04   1.12e+04      2.358      0.038    1763.935    5.13e+04
==============================================================================
Omnibus:                       13.955   Durbin-Watson:                   0.744
Prob(Omnibus):                  0.001   Jarque-Bera (JB):                9.446
Skew:                           1.557   Prob(JB):                      0.00889
Kurtosis:                       5.781   Cond. No.                         21.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

swing


D:\Anaconda\envs\dart\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     cr   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     16.19
Date:                Tue, 12 Sep 2023   Prob (F-statistic):            0.00200
Time:                        18:11:54   Log-Likelihood:                -104.87
No. Observations:                  13   AIC:                             213.7
Df Residuals:                      11   BIC:                             214.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -620.9022    378.243     -1.642      0.129   -1453.410     211.605
te          1.184e+04   2943.105      4.024      0.002    5365.485    1.83e+04
==============================================================================
Omnibus:                        4.894   Durbin-Watson:                   0.781
Prob(Omnibus):                  0.087   Jarque-Bera (JB):                1.939
Skew:                           0.803   Prob(JB):                        0.379
Kurtosis:                       3.999   Cond. No.                         12.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

sz


D:\Anaconda\envs\dart\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     cr   R-squared:                       0.492
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     10.66
Date:                Tue, 12 Sep 2023   Prob (F-statistic):            0.00753
Time:                        18:11:54   Log-Likelihood:                -69.281
No. Observations:                  13   AIC:                             142.6
Df Residuals:                      11   BIC:                             143.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.4219     20.900      1.647      0.128     -11.579      80.423
te           369.5957    113.191      3.265      0.008     120.464     618.728
==============================================================================
Omnibus:                       15.387   Durbin-Watson:                   2.568
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.916
Skew:                           1.413   Prob(JB):                      0.00157
Kurtosis:                       6.982   Cond. No.                         7.65
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""